In [1]:
import ray

In [2]:
# 라이브러리 임포트
import os
import time
import json
import datetime
import numpy as np
import pandas as pd
from fbprophet import Prophet
import pickle
import copy

In [3]:
%%time
submit = json.load(open('./data/sample_submission/sample_submission.json', 'r', encoding='utf8')) 
df_dic = pickle.load(open('./mydata/df_dic.plk','rb'))
plc_lst = pickle.load(open('./mydata/plc_lst.plk','rb'))
fct_lst = ['pH', 'COD', 'SS', 'N', 'P', 'T']

CPU times: user 749 ms, sys: 115 ms, total: 864 ms
Wall time: 865 ms


In [4]:
%%time
''' fillna : 단순 missing value 채움'''
for plc in plc_lst:
    df = df_dic[plc]
    df = df.fillna(method='ffill')
    df = df.fillna(method='bfill')    
    df_dic[plc] = df.dropna()
    
    if df_dic[plc].isna().any().any():
        print(plc)

CPU times: user 2.19 s, sys: 387 µs, total: 2.19 s
Wall time: 2.19 s


In [5]:
# Prophet 예측을 위한 dataframe 포맷 제작
future = pd.DataFrame([str(x)[:10] for x in list(pd.date_range(start='2018-02-01', end='2019-12-31', inclusive="both"))], columns=['ds'])
future['y'] = np.nan
#future

In [10]:
@ray.remote
def pred_all_columns(plc_df,i,fct_lst):
    
    res_list = []
    
    for j in fct_lst: # 피쳐 리스트
        fct_df = plc_df[['ds', j]]   # 날짜 - 피쳐1 조합임.
        fct_df = fct_df.rename(columns={j:'y'}) # 피쳐1을 y로 바꾼다.
        model = Prophet(changepoint_prior_scale = 0.01)
        model.fit(fct_df)
        forecast = model.predict(future)
        forecast['ds'] = forecast['ds'].apply(lambda x : str(x).split(' ')[0].split('-')[0]+str(x).split(' ')[0].split('-')[1]+str(x).split(' ')[0].split('-')[2])
        forecast = forecast[['ds', 'yhat']]
        
        #for index, k in enumerate(list(forecast['ds'])):
        #    submit[i][forecast.loc[index, 'ds']][j] = np.float(forecast[forecast['ds']==k]['yhat'])
            
        res_list.append( (i,j,forecast) )
    return res_list

(pred_all_columns pid=8966)      699       5235.07    7.1417e-06       64.3553      0.2206           1     1137   
(pred_all_columns pid=8966)     Iter      log prob        ||dx||      ||grad||       alpha      alpha0  # evals  Notes 
(pred_all_columns pid=8965)      199       8635.38   0.000539778       328.743      0.4166      0.4166      236   
(pred_all_columns pid=8965)     Iter      log prob        ||dx||      ||grad||       alpha      alpha0  # evals  Notes 
(pred_all_columns pid=8954)     1799       7697.59   6.97367e-05       88.5833           1           1     2511   
(pred_all_columns pid=8954)     Iter      log prob        ||dx||      ||grad||       alpha      alpha0  # evals  Notes 
(pred_all_columns pid=8962)      791        4471.9   0.000500178       178.975   2.966e-06       0.001     1214  LS failed, Hessian reset 
(pred_all_columns pid=8976)      295       4476.55   2.25419e-07       52.1773      0.3428      0.3428      516   
(pred_all_columns pid=8976) Optimization 

(pred_all_columns pid=8976) /ssd/jaewoo/anaconda3/envs/ts/lib/python3.9/site-packages/fbprophet/forecaster.py:891: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
(pred_all_columns pid=8976)   components = components.append(new_comp)


(pred_all_columns pid=8943)      337       7355.98   7.89881e-05       63.6363   1.207e-06       0.001      463  LS failed, Hessian reset 
(pred_all_columns pid=8937)      698       6395.52   2.34864e-05       57.6969    3.57e-07       0.001     1154  LS failed, Hessian reset 
(pred_all_columns pid=8937)      699       6395.52   8.16889e-06       48.2356           1           1     1155   
(pred_all_columns pid=8937)     Iter      log prob        ||dx||      ||grad||       alpha      alpha0  # evals  Notes 
(pred_all_columns pid=8940)       99       4538.59      0.012968        108.33           1           1      109   
(pred_all_columns pid=8940)     Iter      log prob        ||dx||      ||grad||       alpha      alpha0  # evals  Notes 
(pred_all_columns pid=8946)      531       6007.48   0.000460116       194.828   3.849e-06       0.001      804  LS failed, Hessian reset 
(pred_all_columns pid=8966)      780       5235.99   0.000246777       123.584   1.097e-06       0.001     1280  

(pred_all_columns pid=8975) /ssd/jaewoo/anaconda3/envs/ts/lib/python3.9/site-packages/fbprophet/forecaster.py:891: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
(pred_all_columns pid=8975)   components = components.append(new_comp)
(pred_all_columns pid=8959) INFO:fbprophet:Disabling daily seasonality. Run prophet with daily_seasonality=True to override this.
(pred_all_columns pid=8959) /ssd/jaewoo/anaconda3/envs/ts/lib/python3.9/site-packages/fbprophet/forecaster.py:891: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
(pred_all_columns pid=8959)   components = components.append(new_comp)
(pred_all_columns pid=8975) /ssd/jaewoo/anaconda3/envs/ts/lib/python3.9/site-packages/fbprophet/forecaster.py:891: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.

(pred_all_columns pid=8965)      299       8636.97    0.00209978        165.24           1           1      353   
(pred_all_columns pid=8965)     Iter      log prob        ||dx||      ||grad||       alpha      alpha0  # evals  Notes 
(pred_all_columns pid=8937)      717       6395.52   1.16188e-07       48.9866     0.06135           1     1187   
(pred_all_columns pid=8937) Optimization terminated normally: 
(pred_all_columns pid=8937)   Convergence detected: relative gradient magnitude is below tolerance
(pred_all_columns pid=8957)     1592       8265.82   5.80874e-05       164.195   7.811e-07       0.001     2545  LS failed, Hessian reset 
(pred_all_columns pid=8957)     1599       8265.82   2.46907e-06       74.0193     0.08798     0.08798     2555   
(pred_all_columns pid=8957)     Iter      log prob        ||dx||      ||grad||       alpha      alpha0  # evals  Notes 
(pred_all_columns pid=8940)      142       4540.24   0.000261982       102.228   3.458e-06       0.001      196  L

...

(pred_all_columns pid=8979) /ssd/jaewoo/anaconda3/envs/ts/lib/python3.9/site-packages/fbprophet/forecaster.py:891: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
(pred_all_columns pid=8979)   components = components.append(new_comp)
(pred_all_columns pid=8938) /ssd/jaewoo/anaconda3/envs/ts/lib/python3.9/site-packages/fbprophet/forecaster.py:891: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
(pred_all_columns pid=8938)   components = components.append(new_comp)
(pred_all_columns pid=8934) INFO:fbprophet:Disabling daily seasonality. Run prophet with daily_seasonality=True to override this.
(pred_all_columns pid=8934) /ssd/jaewoo/anaconda3/envs/ts/lib/python3.9/site-packages/fbprophet/forecaster.py:891: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.

(pred_all_columns pid=8965)      690        8639.3   1.93485e-05       112.506   1.219e-07       0.001     1012  LS failed, Hessian reset 
(pred_all_columns pid=8956)      285        3191.1    0.00016723       84.3936    2.46e-06       0.001      466  LS failed, Hessian reset 
(pred_all_columns pid=8956)      299       3191.11   9.27764e-06       82.8187      0.1486      0.5776      486   
(pred_all_columns pid=8956)     Iter      log prob        ||dx||      ||grad||       alpha      alpha0  # evals  Notes 
(pred_all_columns pid=8974)      399       6529.42   0.000488718       90.5632           1           1      460   
(pred_all_columns pid=8974)     Iter      log prob        ||dx||      ||grad||       alpha      alpha0  # evals  Notes 
(pred_all_columns pid=8979)       99       7602.48    0.00199948       323.196     0.07955           1      116   
(pred_all_columns pid=8979)     Iter      log prob        ||dx||      ||grad||       alpha      alpha0  # evals  Notes 
(pred_all_columns

(pred_all_columns pid=8963) INFO:fbprophet:Disabling daily seasonality. Run prophet with daily_seasonality=True to override this.
(pred_all_columns pid=8963) /ssd/jaewoo/anaconda3/envs/ts/lib/python3.9/site-packages/fbprophet/forecaster.py:891: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
(pred_all_columns pid=8963)   components = components.append(new_comp)
(pred_all_columns pid=8954) /ssd/jaewoo/anaconda3/envs/ts/lib/python3.9/site-packages/fbprophet/forecaster.py:891: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
(pred_all_columns pid=8954)   components = components.append(new_comp)
(pred_all_columns pid=8954) /ssd/jaewoo/anaconda3/envs/ts/lib/python3.9/site-packages/fbprophet/forecaster.py:891: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.

(pred_all_columns pid=8965)      774       8639.41   3.03414e-06       58.1192   4.967e-08       0.001     1193  LS failed, Hessian reset 
(pred_all_columns pid=8965)      781       8639.41   2.76523e-07       53.3708      0.2005           1     1204   
(pred_all_columns pid=8965) Optimization terminated normally: 
(pred_all_columns pid=8965)   Convergence detected: relative gradient magnitude is below tolerance
(pred_all_columns pid=8968)      401        4166.4    0.00159028        116.81   1.564e-05       0.001      608  LS failed, Hessian reset 
(pred_all_columns pid=8978)      299        5863.2   0.000454465       84.8725      0.5859      0.5859      417   
(pred_all_columns pid=8978)     Iter      log prob        ||dx||      ||grad||       alpha      alpha0  # evals  Notes 
(pred_all_columns pid=8949)      199       5254.13      0.105706       608.557           1           1      230   
(pred_all_columns pid=8949)     Iter      log prob        ||dx||      ||grad||       alpha     

(pred_all_columns pid=8953) INFO:fbprophet:Disabling daily seasonality. Run prophet with daily_seasonality=True to override this.
(pred_all_columns pid=8953) /ssd/jaewoo/anaconda3/envs/ts/lib/python3.9/site-packages/fbprophet/forecaster.py:891: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
(pred_all_columns pid=8953)   components = components.append(new_comp)
(pred_all_columns pid=8936) INFO:fbprophet:Disabling daily seasonality. Run prophet with daily_seasonality=True to override this.
(pred_all_columns pid=8936) /ssd/jaewoo/anaconda3/envs/ts/lib/python3.9/site-packages/fbprophet/forecaster.py:891: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
(pred_all_columns pid=8936)   components = components.append(new_comp)
(pred_all_columns pid=8976) INFO:fbprophet:Disabling daily seasonality. Run prophet with daily_seasona

In [ ]:
fct_lst_ray = ray.put(fct_lst)
#submit_ray = ray.put(submit)

In [ ]:
task_list = [ pred_all_columns.remote(df_dic[x], x ,fct_lst_ray) for x in plc_lst ]

In [13]:
%%time
results = ray.get(task_list)

CPU times: user 384 ms, sys: 0 ns, total: 384 ms
Wall time: 383 ms


---
## 결과 쓰기

In [18]:
@ray.remote
def create_sub(submit, results_partial):
    for item in results_partial:
        for x in item:
            i = x[0]
            j = x[1]
            forecast = x[2]
            for index, k in enumerate(list(forecast['ds'])):
                submit[i][forecast.loc[index, 'ds']][j] = np.float(forecast[forecast['ds']==k]['yhat'])

In [19]:
submit_ray = ray.put(submit)

In [ ]:
%%time
task_list = [ create_sub.remote(submit, x) for x in results_partial ]
ray.get(task_list)

In [14]:
for item in results:
    for x in item:
        i = x[0]
        j = x[1]
        forecast = x[2]
        for index, k in enumerate(list(forecast['ds'])):
            submit[i][forecast.loc[index, 'ds']][j] = np.float(forecast[forecast['ds']==k]['yhat'])

/tmp/ipykernel_8785/3712719606.py:7: DeprecationWarning: `np.float` is a deprecated alias for the builtin `float`. To silence this warning, use `float` by itself. Doing this will not modify any behavior and is safe. If you specifically wanted the numpy scalar type, use `np.float64` here.
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations
  submit[i][forecast.loc[index, 'ds']][j] = np.float(forecast[forecast['ds']==k]['yhat'])


In [16]:
# 추론 파일 저장
with open('./sub/bs_fillna_cp0_01ray.json', 'w', encoding='utf8') as f:
    json.dump(submit, f, ensure_ascii=False)

In [17]:
len(results)

545